In [1]:
topics={ 264014: {'title': "how long is life cycle of flea"}}

In [2]:
from mugi import generate_pseudo_references, ensure_model_loaded
# from openai import OpenAI
import os


In [3]:
model_name = "HuggingFaceTB/cosmo-1b"
ensure_model_loaded(model_name)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.01G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# openai_key =  os.environ.get('OPENAI_KEY')
# client = OpenAI(api_key=openai_key)
new_topics = generate_pseudo_references('l', topics, generated_document_num=3)